## Load Libraries

In [ ]:
# install 
!pip install datasets transformers numpy sentencepiece accelerate -U evaluate

In [2]:
import os
import random
import numpy as np
import pandas as pd

import torch
from torch.utils.data import Dataset, DataLoader

import evaluate
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments, Trainer

from sklearn.model_selection import train_test_split

from tokenization_kobert import KoBertTokenizer

## Set Hyperparameters

In [3]:
SEED = 456 # random seed 고정 (재현성을 위해)
random.seed(SEED)
np.random.seed(SEED) 
torch.manual_seed(SEED) 
torch.cuda.manual_seed(SEED) 
torch.cuda.manual_seed_all(SEED) 

In [7]:
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu') # 디바이스 설정
DEVICE

device(type='cuda')

In [4]:
# 경로 설정
BASE_DIR = os.getcwd() # 현재 디렉토리
DATA_DIR = os.path.join(BASE_DIR, '../data') 
OUTPUT_DIR = os.path.join(BASE_DIR, '../output')

## Load Tokenizer and Model

아래 조건에 맞춰 스스로 코드를 짜봅시다!
- 사용할 모델 : https://huggingface.co/monologg/kobert
- tokenizer는 ./SK_Day2/text-cls-ynat/tokenization_kobert.py 를 사용
- 모델 로드에는 AutoModelForSequenceClassification 클래스를 사용

## Define Dataset

- pd.read_csv 를 사용하여 데이터 불러오기
- 불러온 데이터를 train/valid 셋으로 분리 (train set : valid set = 7:3)

- BERTDataset 클래스 완성하기

In [7]:
class BERTDataset(Dataset):
    def __init__(self, data, tokenizer):

        
    
    def __getitem__(self, idx): 
        
    
    def __len__(self):
       

In [8]:
data_train = # train 데이터셋 생성
data_valid = # valid 데이터셋 생성

In [9]:
data_collator =  # padding을 위한 data_collator 생성

## Define Metric

In [10]:
f1 = evaluate.load('f1') # f1 score를 계산하기 위한 함수
def compute_metrics(eval_pred):
    predictions, labels = eval_pred # model의 output과 실제 label을 받음
    predictions = np.argmax(predictions, axis=1) # model의 output에서 가장 높은 값을 가진 index를 예측값으로 사용
    return f1.compute(predictions=predictions, references=labels, average='macro') # f1 score 계산, macro: label별 f1 score의 평균


## Train Model

In [11]:
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    overwrite_output_dir=True,
    do_train=True,
    do_eval=True,
    do_predict=True,
    logging_strategy='steps',
    evaluation_strategy='steps',
    save_strategy='steps',
    logging_steps=100,
    eval_steps=100,
    save_steps=100,
    save_total_limit=2,
    learning_rate= 2e-05,
    adam_beta1 = 0.9,
    adam_beta2 = 0.999,
    adam_epsilon=1e-08,
    weight_decay=0.01,
    lr_scheduler_type='linear',
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    load_best_model_at_end=True,
    metric_for_best_model='eval_f1',
    greater_is_better=True, 
    seed=SEED,
    auto_find_batch_size=True 
)

- Trainer의 인자들을 넣어봅시다!

In [24]:
trainer = Trainer()

- 훈련을 시켜봅시다! 
- (훈련 시간이 너무 오래걸려서 로그가 찍히는 것만 확인하고 중지 시키시면 됩니다!)

## Evaluate Model

- test set 에 대해 예측을 해봅시다!

In [5]:
BASE_DIR = os.getcwd() # 현재 디렉토리
dataset_test = # 데이터 읽어오기

In [ ]:
model_name = ## 채우기
tokenizer = ## 채우기
model = ## 채우기
model.eval() # model을 evaluation 모드로 변경
preds = [] 
for idx, sample in dataset_test.iterrows():
    inputs = ## 채우기
    with torch.no_grad(): # gradient 계산 비활성화
        logits = ## 채우기  
        pred = ## 채우기 
        preds.extend(pred) 

- 결과 csv 파일로 저장

In [11]:
dataset_test['target'] = preds
BASE_DIR = os.getcwd()
dataset_test.to_csv(os.path.join(BASE_DIR, 'test_output.csv'), index=False)